# MCIR Reconstruction of StarVibe data

In [4]:
import sirf.Gadgetron as pMR

import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt

## Preparation
This code works with SIRF-branch ignore-acq


Currently there is a bug in the coil sensitivity map estimation. See https://github.com/SyneRBI/SIRF/issues/1221

To solve this temporarily:
Remove "&& object_mask[i]" from the code here 
https://github.com/SyneRBI/SIRF/blob/1bb1b111a80f48df5e491a8ee9201715d4e9db42/src/xGadgetron/cGadgetron/gadgetron_data_containers.cpp#L2811C33-L2811C48

and compile code again.

## Define trajectory 

StarVibe uses a stack-of-stars trajectory with a Golden angle increment. 

In [5]:
# Trajectory for Golden angle radial acquisitions
def calc_rad_traj_golden(ad):
    dims = ad.dimensions()
    kx = np.linspace(-dims[2]//2, dims[2]//2, dims[2])
    ky = ad.get_ISMRMRD_info('kspace_encode_step_1')
    
    # Define angle with Golden angle increment
    angRad = ky * np.pi * 0.618034

    # Normalise radial points between [-0.5 0.5]
    krad = kx
    krad = krad / (np.amax(np.abs(krad)) * 2)

    # Calculate trajectory
    rad_traj = np.zeros((dims[2], dims[0], 2), dtype=np.float32)
    rad_traj[:, :, 0] = krad.reshape(-1, 1) * np.cos(angRad)
    rad_traj[:, :, 1] = krad.reshape(-1, 1) * np.sin(angRad)
    rad_traj = np.moveaxis(rad_traj, 0, 1)
    return(rad_traj)

## Reconstruct motion corrupted data set

In [8]:
# Read in data

filename = '/data/Paul/' + 'meas_MID00037_FID08328_Tho_starvibe_BodyCOMPASS_USER_noFS_kz38.h5'

# Nothing to ignore except for noise samples
ignored = pMR.IgnoreMask() 
acq_data = pMR.AcquisitionData(filename, False, ignored=ignored)
acq_data.sort_by_time()

# Get k-space dimensions and encoding limits
print(f'Dimensions of acq data: {acq_data.dimensions()}')

encode_step_1 = acq_data.get_ISMRMRD_info('kspace_encode_step_1')
print(f'Angle index goes from {np.min(encode_step_1)} to {np.max(encode_step_1)}')
encode_step_2 = acq_data.get_ISMRMRD_info('kspace_encode_step_2')
print(f'Slice index goes from {np.min(encode_step_2)} to {np.max(encode_step_2)}')

reading from /data/Paul/meas_MID00037_FID08328_Tho_starvibe_BodyCOMPASS_USER_noFS_kz38.h5 using ignore mask 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0000 0100 0000 0000 0000 0000 
Started reading acquisitions from /data/Paul/meas_MID00037_FID08328_Tho_starvibe_BodyCOMPASS_USER_noFS_kz38.h5
0%..10%..20%..31%..40%..50%..61%..70%..80%..90%..
Finished reading acquisitions from /data/Paul/meas_MID00037_FID08328_Tho_starvibe_BodyCOMPASS_USER_noFS_kz38.h5
Dimensions of acq data: (60800, 18, 384)
Angle index goes from 0 to 1599
Angle index goes from 0 to 37


In [9]:
# Now we create the trajectory and set it
ktraj = calc_rad_traj_golden(acq_data)
pMR.set_radial2D_trajectory(acq_data, ktraj)

In [10]:
# Calculate the coil sensitivity maps
csm = pMR.CoilSensitivityData()
csm.smoothness = 100
csm.calculate(acq_data)
csm_arr = csm.as_array()
print(csm_arr.shape)

In [ ]:
# Visualise coil maps
fig, ax = plt.subplots(4,5) # 4 coils, 5 slices
slice_step = csm_arr.shape[0]//ax.shape[1]
for cnd in range(ax.shape[0]):
    for snd in range(ax.shape[1]):
        ax[cnd,snd].imshow(np.abs(csm_arr[slice_step*snd,:,cnd,:]))

In [ ]:
# Define acquisition model
E_sos = pMR.AcquisitionModel(acqs=acq_data, imgs=csm)
E_sos.set_coil_sensitivity_maps(csm)

In [ ]:
# Reconstruct images
im_data = E_sos.inverse(acq_data)

In [ ]:
# Visualise images
fig, ax = plt.subplots(2,5) # 2 views, 5 slices
z_step = csm_arr.shape[0]//ax.shape[1]
y_step = csm_arr.shape[1]//ax.shape[1]
for ind in range(ax.shape[1]):
    ax[0,ind].imshow(np.abs(csm_arr[z_step*ind,:,0,:]))
    ax[1,ind].imshow(np.abs(csm_arr[:,y_step*ind,0,:]))

## Reconstruction of motion resolved images

### ToDo

- Find k-space center (kz=kx=0) values for each angle
- Extract k-space values and apply filter to get smooth breathing signal (ideally we would use a PCA analysis over the coil dimension)
- Split data into different respiratory bins based on motion signal
- Reconstruct motion resolved data

Further info can be found here:
https://github.com/SyneRBI/SIRF-Exercises/blob/MCIR_CIL/notebooks/MR/h_mr_mcir_grpe.ipynb